


# <img src="assets/voxel51_logo.png" alt="Image2" width="40"/> FiftyOne + Vector Search
This notebook demonstrates how to build a complete visual search workflow using **FiftyOne** and **Vector Search**.

You will learn how to:
- Load and index embeddings using FiftyOne
- Query by image and text
- Visualize results in the FiftyOne App

🧠 This integration helps you scale visual search over large datasets with a cloud-native vector database.

👉 As an example of vector search, see this official documentation [FiftyOne + Mosaic AI docs](https://docs.voxel51.com/integrations/mosaic.html)


<img src="assets/mosaic_fiftyone_recipe.png" alt="Image2" width="600"/>

https://github.com/user-attachments/assets/2f5f21b3-5f42-4ab5-8e29-e1cac3e8eeb1

In [ ]:
# Install necessary packages
#!pip install fiftyone torch torchvision python-dotenv mlflow umap-learn


Wait until this endpoint is ready, any action before that can create a 500 or 400 HTTP Error.

## 📁 Load the BDD100K Dataset and Launch FiftyOne
We will use the `BDD100K` dataset from HuggingFace Hub.

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

import fiftyone.utils.huggingface as fouh # Hugging Face integration

import os

# Increase both connection and read timeout values (in seconds)
# os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "60"  # default is 10
# os.environ["HF_HUB_ETAG_TIMEOUT"] = "30"      # metadata fetch timeout
# dataset = fouh.load_from_hub("dgural/bdd100k", persistent=True) #, overwrite=True)

# Define the new dataset name
dataset_name = "dgural/bdd100k"

# Check if the dataset exists
if dataset_name in fo.list_datasets():
    print(f"Dataset '{dataset_name}' exists. Loading...")
    dataset = fo.load_dataset(dataset_name)
else:
    print(f"Dataset '{dataset_name}' does not exist. Creating a new one...")
    # Clone the dataset with a new name and make it persistent
    #dataset = dataset.clone(dataset_name, persistent=True)



Dataset 'dgural/bdd100k' exists. Loading...


In [3]:
print(fo.list_datasets())

['2025.03.25.13.07.42', '2025.03.25.13.08.53', '2025.04.22.12.27.21', 'ADL_Fall_Videos_Eval', 'AnomalyMerged_MVTec_ViSA', 'Voxel51/mvtec-ad', 'anomaly_predictions_grouped', 'anomaly_predictions_grouped_carpet', 'coffee_FO', 'coffee_FO_SAM2_process', 'coffee_FO_geolocation', 'dgural/bdd100k', 'mvtec-ad_1', 'mvtec-ad_2', 'mvtec-ad_3', 'mvtec-ad_4', 'mvtec-ad_5', 'mvtec-ad_6', 'mvtec-ad_ad-1', 'mvtec-ad_demo', 'mvtec-ad_no_categories', 'mvtec-bottle', 'mvtec-bottle_2', 'mvtec-carpet-1', 'mvtec-screw', 'mvtecad2', 'mvtecad2_grouped', 'pjramg/my_colombian_coffe_FO', 'potato_mvtec', 'ucf101-test']


In [4]:
session = fo.launch_app(dataset, port=5151, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


## Using the SKLearn backend (By default)
By default, calling ```compute_similarity()``` or ```sort_by_similarity()``` will use an sklearn backend.
To use the Mosaic backend, simply set the optional backend parameter of ```compute_similarity()``` to ```mosaic```:

## 🧠 Compute Embeddings and Index with SKLearn
Now we compute a similarity index using the Mosaic backend. This will:
- Use a CLIP model to generate embeddings
- Compute visualization
- Compute Similarity
- Text promt the dataset, create a view, find mistakes.

In [5]:
model = foz.load_zoo_model("clip-vit-base32-torch")
embeddings = dataset.compute_embeddings(model, embeddings_field = "clip_embeddings")

sklear_idx = fob.compute_similarity(dataset, brain_index = "test_idx", model = "clip-vit-base32-torch", embeddings = "clip_embeddings")

 100% |█████████████| 10000/10000 [5.1m elapsed, 0s remaining, 32.9 samples/s]      


In [6]:
session = fo.launch_app(dataset, port=5151, auto=False)


Session launched. Run `session.show()` to open the App in a cell output.


In [7]:
# Query by first image sample
query = dataset.first().id
view = dataset.sort_by_similarity(query, k=10)
session.view = view

/Users/paularamos/Documents/GitHub/awesome-fiftyone/DB_Mosaic_AI_FiftyOne/fiftyone_DB_env/lib/python3.13/site-packages/fiftyone/core/stages.py:8965: UserWarning: Multiple similarity runs found; using 'similarity_index'
  warnings.warn(msg)


Generating index for 10000 embeddings...
Index complete


In [8]:
dataset.reload()

print(dataset)
print(dataset.first())

Name:        dgural/bdd100k
Media type:  image
Num samples: 10000
Persistent:  True
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    detections:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    polylines:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Polylines)
    weather:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    timeofday:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    scene:            fiftyone.core.fields.EmbeddedDocumentField(fifty

In [10]:
# Query by text prompt
# DETECTIONS: bike  bus  car  motor  person  rider  traffic light  traffic sign  train  truck
# WEATHER: overcast  foggy  rainy  snowy  undefined  partly cloudy  clear
# SCENE: city street  gas stations  highway  parking lot  residential  tunnel 
# TIME OF DAY: daytime  night  dawn/dusk

query_txt = "foggy day" 
view_txt = dataset.sort_by_similarity(query_txt, k=50)
session.view = view_txt

/Users/paularamos/Documents/GitHub/awesome-fiftyone/DB_Mosaic_AI_FiftyOne/fiftyone_DB_env/lib/python3.13/site-packages/fiftyone/core/stages.py:8965: UserWarning: Multiple similarity runs found; using 'similarity_index2'
  warnings.warn(msg)


### 📦 Install `umap-learn`
`umap-learn` is required to visualize high-dimensional embeddings in 2D or 3D.

```bash
pip install umap-learn
```

In [ ]:
# Compute visualization
fob.compute_visualization(dataset, embeddings="clip_embeddings", brain_key="bdd100k_key_")

Generating visualization...


/Users/paularamos/Documents/GitHub/awesome-fiftyone/DB_Mosaic_AI_FiftyOne/fiftyone_DB_env/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


UMAP( verbose=True)
Sat Apr 26 00:53:38 2025 Construct fuzzy simplicial set
Sat Apr 26 00:53:38 2025 Finding Nearest Neighbors
Sat Apr 26 00:53:38 2025 Building RP forest with 10 trees
Sat Apr 26 00:53:40 2025 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	 5  /  13
	Stopping threshold met -- exiting after 5 iterations
Sat Apr 26 00:53:43 2025 Finished Nearest Neighbor Search
Sat Apr 26 00:53:44 2025 Construct embedding


Epochs completed:  10%| ▉          48/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs


...

	completed  300  /  500 epochs
	completed  350  /  500 epochs


Epochs completed:  92%| █████████▏ 460/500 [00:01]

	completed  400  /  500 epochs
	completed  450  /  500 epochs


Epochs completed: 100%| ██████████ 500/500 [00:01]


Sat Apr 26 00:53:46 2025 Finished embedding


## Cleanup (Optional)

In [ ]:
# Delete Mosaic index and run record
# index.cleanup()
# dataset.delete_brain_run("index")
# dataset.delete_brain_runs()